In [58]:
import pandas as pd
import dateutil
import warnings
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

pd.set_option('display.max_colwidth', 1000)
warnings.filterwarnings('ignore')

In [59]:
dataset_df = pd.read_csv('../data/chatgpt_daily_tweets.csv')
dataset_en_df = dataset_df[dataset_df['lang'] == 'en']
dataset_en_df

,tweet_id,tweet_created,tweet_extracted,text,lang,user_id,user_name,user_username,user_location,user_description,user_created,user_followers_count,user_following_count,user_tweet_count,user_verified,source,retweet_count,like_count,reply_count,impression_count
2,1.6427385624866693e+18,2023-04-03 03:59:28+00:00,2023-04-08 01:06:52.504868,RT @DarrellLerner: ChatGPT Plugins are the fastest way to get rich in 2023. \n\nI’ve created a step-by-step guide showing you how to earn $10…,en,2383245894,pk,pradeep42329225,India,💐💐।।जय श्री महाकाल।।💐💐,2014-03-11 06:04:10+00:00,269.0,4141.0,3816.0,False,NaN,628.0,0.0,0.0,0.0
3,1.6429198880616448e+18,2023-04-03 15:59:59+00:00,2023-04-08 01:07:04.742617,Get an intelligent chatbot for your website in minutes with Chatbase AI. Train ChatGPT on your data and let it answer any question your users have. Simply upload a document or link and add the chat widget - it's that easy!\nMake Money using AI: https://t.co/yLHEqn4w9T https://t.co/ba54JvoRsM,en,1633040597782081537,AR Leyva,ArrheniusLey,United Kingdom,Passionate about AI and its potential to transform the business landscape and shape the future. https://t.co/c3UpfbLbqH,2023-03-07 09:43:36+00:00,264.0,24.0,198.0,False,NaN,0.0,0.0,0.0,58.0
4,1.642708351690711e+18,2023-04-03 01:59:25+00:00,2023-04-08 01:06:50.638068,"🔥Hey Guys, #ZenithSwap has launched at just $ 55,000 USD Marketcap. The ChatGPT of DEX - Reimagining DeFi with AI-Powered Yield Farming. Now at 4X. Lot of up potential at such low marketcap.🔥😇 $ARB $ZSP #Arbitrum https://t.co/9VWYtYzAJD",en,1311403370670960640,Human Being 🇨🇳🇸🇬🇻🇳,KiarostamiBeing,"Chicago, IL",Freedom is the Recognition of Necessity • Market Socialism • Cheng Enfu Stan • Empiricism is the Science of Economics || Stay Humble ✌️,2020-09-30 20:32:00+00:00,447.0,419.0,12949.0,False,NaN,0.0,0.0,0.0,0.0
5,1.6428593561893274e+18,2023-04-03 11:59:28+00:00,2023-04-08 01:07:00.375167,RT @sinsonetwork: Now! Join #SINSO DataLand^ChatGPT #Airdrop!\n📅3.23-4.6\n📌Tasks\n①Log in to&lt;https://t.co/Hlwqa7HG40&gt;\n②Try SINSO #ChatGPT&amp; twe…,en,462142717,MokoHaram,holymoko,Zambia mufulira,Public faker🍂,2012-01-12 16:25:12+00:00,2286.0,982.0,20438.0,False,NaN,270.0,0.0,0.0,0.0
9,1.6428744495305933e+18,2023-04-03 12:59:26+00:00,2023-04-08 01:07:01.437161,"The plagiarism detector will introduce its #AI detection tool tomorrow, hoping to protect academic integrity in a post-#ChatGPT world. \n\nThe speedy launch and lack of an opt-out have #academics worried.\n\nAn important piece @liamhknox for @insidehighered \n\nhttps://t.co/pq7DB5r9An",en,896094324341039104,Dr. Susan D'Agostino,susan_dagostino,"New Hampshire, USA",Tech reporter @insidehighered. Mathematician. Bylines @WashingtonPost @TheAtlantic @QuantaMagazine @WIRED @BulletinAtomic @Nature @NPR @BBC,2017-08-11 19:41:50+00:00,4444.0,1924.0,3320.0,True,NaN,7.0,22.0,1.0,5769.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20996,1648959550253330432,2023-04-20 07:59:27+00:00,2023-04-22 21:54:52.216843,@voltatousc @elonmusk @alx For good reason. It was ChatGPT nonsense. Should have verified elsewhere. I appreciate the correction. https://t.co/HgEWxK64tc,en,96104210,Gaetano D'Orsi,Tudor1944,Italy,Solution Engineer @Hyland / @Alfresco | \nOpen Source | Cloud Architectures,2009-12-11 10:56:46+00:00,63.0,224.0,418.0,False,NaN,0.0,1.0,0.0,56.0
20997,1649095523784327168,2023-04-20 16:59:46+00:00,2023-04-22 21:55:01.420991,"RT @ddskier: 📣TOMORROW! Join me and my smart, insightful cohorts for a great session on ""Business Applications of Generative AI"". We'll tal…",en,1499528886018658311,Datadevstuff,datadevstuff,NaN,Tweeting & retweeting interesting tech including #data #python #datascience #SQL #devtools #Snowflake #cloud #dataanalytics #Scala #AI,2022-03-03 23:35:48+00:00,44.0,143.0,453.0,False,NaN,2.0,0.0,0.0,0.0
20998,1648944424339079168,2023-04-20 06:59:21+00:00,2023-04-22 21:54:51.264432,@jirannni @its_fanuel This can't be chatgpt,en,1440707901048307723,W

In [60]:
def clean_data(dataset_df: pd.DataFrame) -> pd.DataFrame:
    try:
        data = dataset_df.copy()
        # Convert the 'date' column to datetime format and remove the timezone information
        data['tweet_created'] = data['tweet_created'].apply(lambda x: dateutil.parser.parse(x))
        data['tweet_created'] = pd.to_datetime(data['tweet_created']).dt.tz_localize(None)

        # data['tweet_extracted'] = data['tweet_extracted'].apply(lambda x: dateutil.parser.parse(x))
        # data['tweet_extracted'] = pd.to_datetime(data['tweet_extracted']).dt.tz_localize(None)

        # Text cleaning
        data['text'] = data['text'].str.lower()

        # Remove twitter handlers
        data['text'] = data['text'].apply(lambda x:re.sub('@[^\s]+','',x))

        # Remove hashtags
        data['text'] = data['text'].apply(lambda x:re.sub(r'\B#\S+','',x))

        # Remove URLS
        data['text'] = data['text'].apply(lambda x:re.sub(r"http\S+", "", x))

        # Remove all the special characters
        data['text'] = data['text'].apply(lambda x:' '.join(re.findall(r'\w+', x)))

        # Remove all single characters
        data['text'] = data['text'].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))

        # Substituting multiple spaces with single space
        data['text'] = data['text'].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

        data = data.rename(columns={
            'text': 'original_tweet',
            'tweet_created': 'datetime'
        })

        return data
    except ... as e:
        # Catch all errors
        print(f'Caught Error: {e}')


In [61]:
data = clean_data(dataset_df)
data.head()

,tweet_id,datetime,tweet_extracted,original_tweet,lang,user_id,user_name,user_username,user_location,user_description,user_created,user_followers_count,user_following_count,user_tweet_count,user_verified,source,retweet_count,like_count,reply_count,impression_count
0,1.642889622681432e+18,2023-04-03 13:59:44,2023-04-08 01:07:02.538242,rt เทคน คฝ กภาษาก บ chatgpt ท ผมลอง ผมลองฝ ก อ งกฤษ ญ ป น อ งกฤษ เยอรม น ใช ว ธ เด ยวก น ได ผลเป นท น าพอใจคร บ เหล อแค,th,4706577259,👷🏼 ♡ #GOT7,BPawarisa1a,ในใจJacksonwang,@JacksonWang852 ➖ รีวิว #รีวิวแบมพี #แบมพีอัพเดท ✨ #แบมพีสะสมแต้มบุญ #แบมอยากแจก,2016-01-04 02:27:33+00:00,1293.0,445.0,87051.0,False,NaN,13640.0,0.0,0.0,0.0
1,1.6428442314496123e+18,2023-04-03 10:59:22,2023-04-08 01:06:59.379927,chatgptをもっと活かせるchrome拡張機能4選,ja,2264288640,ミミズクりんゆ,DRVO_Project,東京←岐阜,料理垢です。ｳｪｯﾌﾞ4年目 発言はファッキン個人の見解 TypeScript/Vue/React/広報/採用 ChatGPT plus プロンプトエンジニア組織や環境を作る人へのシフト 音楽活動：@DrvoProject,2013-12-27 12:39:07+00:00,7878.0,4941.0,76597.0,False,NaN,0.0,0.0,0.0,290.0
2,1.6427385624866693e+18,2023-04-03 03:59:28,2023-04-08 01:06:52.504868,rt chatgpt plugins are the fastest way to get rich in 2023ve createdstep by step guide showing you how to earn 10,en,2383245894,pk,pradeep42329225,India,💐💐।।जय श्री महाकाल।।💐💐,2014-03-11 06:04:10+00:00,269.0,4141.0,3816.0,False,NaN,628.0,0.0,0.0,0.0
3,1.6429198880616448e+18,2023-04-03 15:59:59,2023-04-08 01:07:04.742617,get an intelligent chatbot for your website in minutes with chatbase ai train chatgpt on your data and let it answer any question your users have simply uploaddocument or link and add the chat widget itthat easy make money using ai,en,1633040597782081537,AR Leyva,ArrheniusLey,United Kingdom,Passionate about AI and its potential to transform the business landscape and shape the future. https://t.co/c3UpfbLbqH,2023-03-07 09:43:36+00:00,264.0,24.0,198.0,False,NaN,0.0,0.0,0.0,58.0
4,1.642708351690711e+18,2023-04-03 01:59:25,2023-04-08 01:06:50.638068,hey guys has launched at just 55 000 usd marketcap the chatgpt of dex reimagining defi with ai powered yield farming now at 4x lot of up potential at such low marketcap arb zsp,en,1311403370670960640,Human Being 🇨🇳🇸🇬🇻🇳,KiarostamiBeing,"Chicago, IL",Freedom is the Recognition of Necessity • Market Socialism • Cheng Enfu Stan • Empiricism is the Science of Economics || Stay Humble ✌️,2020-09-30 20:32:00+00:00,447.0,419.0,12949.0,False,NaN,0.0,0.0,0.0,0.0


In [62]:
def label_sentiment(x:float):
    if x < -0.05 : return 'negative'
    if x > 0.35 : return 'positive'
    return 'neutral'

In [74]:
def perform_feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    # Sentiment intensity analyzer
    # sia = SIA()
    # data['sentiment'] = [sia.polarity_scores(x)['compound'] for x in data['text']]
    # data['overall_sentiment'] = data['sentiment'].apply(label_sentiment)

    # Feature extraction from tweet text
    data['words'] = data['original_tweet'].apply(lambda x:re.findall(r'\w+', x ))
    data['words_count'] = data['words'].apply(len)
    data['sentence_length'] = data['original_tweet'].apply(len)
    data['hour'] = data['datetime'].apply(lambda x: x.hour)
    data['date'] = data['datetime'].apply(lambda x: x.date())
    data['month'] = data['datetime'].apply(lambda x: x.month)
    data['year'] = data['datetime'].apply(lambda x: x.year)

    return data
    

In [75]:
data_with_features = perform_feature_engineering(data)
data_with_features.head()

,tweet_id,datetime,tweet_extracted,original_tweet,lang,user_id,user_name,user_username,user_location,user_description,...,like_count,reply_count,impression_count,words,words_count,sentence_length,hour,date,month,year
0,1.642889622681432e+18,2023-04-03 13:59:44,2023-04-08 01:07:02.538242,rt เทคน คฝ กภาษาก บ chatgpt ท ผมลอง ผมลองฝ ก อ งกฤษ ญ ป น อ งกฤษ เยอรม น ใช ว ธ เด ยวก น ได ผลเป นท น าพอใจคร บ เหล อแค,th,4706577259,👷🏼 ♡ #GOT7,BPawarisa1a,ในใจJacksonwang,@JacksonWang852 ➖ รีวิว #รีวิวแบมพี #แบมพีอัพเดท ✨ #แบมพีสะสมแต้มบุญ #แบมอยากแจก,...,0.0,0.0,0.0,"[rt, เทคน, คฝ, กภาษาก, บ, chatgpt, ท, ผมลอง, ผมลองฝ, ก, อ, งกฤษ, ญ, ป, น, อ, งกฤษ, เยอรม, น, ใช, ว, ธ, เด, ยวก, น, ได, ผลเป, นท, น, าพอใจคร, บ, เหล, อแค]",33,119,13,2023-04-03,4,2023
1,1.6428442314496123e+18,2023-04-03 10:59:22,2023-04-08 01:06:59.379927,chatgptをもっと活かせるchrome拡張機能4選,ja,2264288640,ミミズクりんゆ,DRVO_Project,東京←岐阜,料理垢です。ｳｪｯﾌﾞ4年目 発言はファッキン個人の見解 TypeScript/Vue/React/広報/採用 ChatGPT plus プロンプトエンジニア組織や環境を作る人へのシフト 音楽活動：@DrvoProject,...,0.0,0.0,290.0,[chatgptをもっと活かせるchrome拡張機能4選],1,27,10,2023-04-03,4,2023
2,1.6427385624866693e+18,2023-04-03 03:59:28,2023-04-08 01:06:52.504868,rt chatgpt plugins are the fastest way to get rich in 2023ve createdstep by step guide showing you how to earn 10,en,2383245894,pk,pradeep42329225,India,💐💐।।जय श्री महाकाल।।💐💐,...,0.0,0.0,0.0,"[rt, chatgpt, plugins, are, the, fastest, way, to, get, rich, in, 2023ve, createdstep, by, step, guide, showing, you, how, to, earn, 10]",22,113,3,2023-04-03,4,2023
3,1.6429198880616448e+18,2023-04-03 15:59:59,2023-04-08 01:07:04.742617,get an intelligent chatbot for your website in minutes with chatbase ai train chatgpt on your data and let it answer any question your users have simply uploaddocument or link and add the chat widget itthat easy make money using ai,en,1633040597782081537,AR Leyva,ArrheniusLey,United Kingdom,Passionate about AI and its potential to transform the business landscape and shape the future. https://t.co/c3UpfbLbqH,...,0.0,0.0,58.0,"[get, an, intelligent, chatbot, for, your, website, in, minutes, with, chatbase, ai, train, chatgpt, on, your, data, and, let, it, answer, any, question, your, users, have, simply, uploaddocument, or, link, and, add, the, chat, widget, itthat, easy, make, money, using, ai]",41,231,15,2023-04-03,4,2023
4,1.642708351690711e+18,2023-04-03 01:59:25,2023-04-08 01:06:50.638068,hey guys has launched at just 55 000 usd marketcap the chatgpt of dex reimagining defi with ai powered yield farming now at 4x lot of up potential at such low marketcap arb zsp,en,1311403370670960640,Human Being 🇨🇳🇸🇬🇻🇳,KiarostamiBeing,"Chicago, IL",Freedom is the Recognition of Necessity • Market Socialism • Cheng Enfu Stan • Empiricism is the Science of Economics || Stay Humble ✌️,...,0.0,0.0,0.0,"[hey, guys, has, launched, at, just, 55, 000, usd, marketcap, the, chatgpt, of, dex, reimagining, defi, with, ai, powered, yield, farming, now, at, 4x, lot, of, up, potential, at, such, low, marketcap, arb, zsp]",34,176,1,2023-04-03,4,2023
